<a href="https://colab.research.google.com/github/41371130H/PL-Repo/blob/main/HW5_%E5%8D%88%E9%A4%90%E8%81%9A%E6%9C%83%E6%B1%BA%E7%AD%96%E5%99%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **🍽️ 用餐決策器**

*歡迎使用本系統！這是一個專為解決「今天吃什麼？」難題而設計的 AI 輔助工具。*

*您可以透過這個網頁介面，一鍵篩選符合您需求的餐廳，並利用 Gemini AI 自動生成幽默的決策建議，同時在地圖上直觀瀏覽所有餐廳位置。*

Google Sheet 連結 ：https://docs.google.com/spreadsheets/d/1Byh4EkK5VyBzc7LAiwMT-pdA2_QlJXy33jaipyMooEg/edit?usp=sharing

---

## 🍽️ 餐廳推薦器
這是本系統的核心功能頁籤，用於篩選並產生推薦。

### **條件篩選**

在啟動推薦前，您需要設定您的偏好：

* **價位過濾 (可複選)**：選擇您能接受的價位 (`100↓`, `200↓`, `200↑`)。若**留空不選**，系統將預設在所有有效價位中進行搜尋。
* **餐別過濾 (可複選)**：選擇您想吃的餐別 (例如：`早餐`, `午餐`...)。若留空，則不過濾餐別。
* **營業日篩選**：選擇您想查詢的日期。預設為 `只看今日營業`，系統會自動判斷；您也可以手動選擇 `週一` 至 `週日` 的特定日期。
* **🚀 開始推薦**：設定好條件後，點擊此按鈕即可開始執行。

### **推薦結果**

點擊按鈕後，右側將會顯示分析結果：

#### **1. 推薦清單 (隨機 3 家)**
* 系統會從所有符合條件的餐廳中，隨機挑選 3 家。
* **推薦結果 (表格)**：以表格形式顯示推薦的餐廳，包含 **店家名稱**、**招牌菜**、**價位**，以及可直接點擊的 **Google map** 連結。

#### **2. 🤖 AI 決策建議**
* 由 Gemini AI (gemini-2.5-pro) 根據推薦清單，即時分析其優缺點（如價格、招牌菜），並提供一份幽默風趣的決策建議，幫助您快速做出最終選擇。

---

## 🗺️ 地圖總覽

這是一個輔助分頁，用來直觀地瀏覽所有餐廳的地理位置與詳細資訊。

### **地圖操作與資訊顯示**

* **動態營業狀態**：地圖上的標記 (Icon) 顏色會根據「今天」的營業狀態自動變化：
    * **藍色**：今日營業
    * **灰色**：今日休息
* **豐富資訊視窗**：點擊任一「餐具」圖示，即可彈出該餐廳的**完整資訊卡**，清楚顯示店家名稱、價位、營業星期、供應餐別等所有細節。
* **小地圖導航 (MiniMap)**：地圖**左下角**提供一個縮略圖 (MiniMap)，讓您在放大檢視時，仍可保持對周邊區域的整體空間感，並快速移動檢視畫面。此功能可切換顯示/隱藏。

### **圖層管理 (Layer Control)**

*此功能位於地圖右上角，您可以透過勾選或取消勾選來控制地圖上顯示的資訊類別。*

* **價位分組圖層**：
    * 可針對 (`100↓`, `200↓`, `200↑`) 三種價位圖層，獨立開關顯示。
    * 透過此功能，您可以一鍵篩選地圖上只顯示特定價格區間的餐廳。
* **底圖切換圖層**：
    * 可選擇不同的地圖樣式，例如：`基本模式 (openstreetmap)`、`簡潔模式 (亮色)`、`暗色模式` 或 `衛星影像`。

---

## ☁️ 雲端資料來源 (後台功能)

系統在每次啟動時，都會從上方的 **Google Sheets** 連結 (`餐廳列表` 工作表) 讀取最新的餐廳資料。

In [38]:
# --- 1. 安裝所需套件 ---
!pip -q install gspread gspread_dataframe google-auth google-auth-oauthlib google-auth-httplib2 \
             gradio pandas beautifulsoup4 google-generativeai python-dateutil folium

In [39]:
# --- 2. 導入所需模組 ---
import os, time, uuid, re, json, datetime
from datetime import datetime as dt, timedelta
from dateutil.tz import gettz
import pandas as pd
import numpy as np
import random
import gradio as gr
import asyncio
import requests
from bs4 import BeautifulSoup
import folium
from folium.plugins import MiniMap
import google.generativeai as genai

# Google Auth & Sheets
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from google.auth import default

# 顯示 Folium 地圖所需
from IPython.display import display

In [40]:
# --- 3. Google 服務認證與設定 ---

# 3.1. Colab 使用者認證
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# 3.2. Gemini API 金鑰設定
from google.colab import userdata
api_key = userdata.get('gemini')
# 使用獲取的金鑰配置 genai
genai.configure(api_key=api_key)
model = genai.GenerativeModel('gemini-2.5-pro')

In [41]:
# --- 4. 載入 Google Sheet 資料 ---

SHEET_URL = "https://docs.google.com/spreadsheets/d/1Byh4EkK5VyBzc7LAiwMT-pdA2_QlJXy33jaipyMooEg/edit?usp=sharing"
WORKSHEET_NAME = "餐廳列表" # 您程式碼一中的工作表名稱
TIMEZONE = "Asia/Taipei"

try:
    # 從 gsheets 的 All-whiteboard-device 載入 sheets
    gsheets = gc.open_by_url(SHEET_URL)
    sheets = gsheets.worksheet(WORKSHEET_NAME).get_all_values()
    # 將 sheets1 資料載入 pd 的 DataFrame 進行分析
    df_restaurants = pd.DataFrame(sheets[1:], columns=sheets[0])
    # 取得最前面的5筆資料
    print(f"--- 成功從 '{WORKSHEET_NAME}' 載入 {len(df_restaurants)} 筆資料 ---")
    print(df_restaurants.head())
except Exception as e:
    print(f"!!! 讀取 Google Sheet 失敗: {e}")

print("\n--- 正在清理資料 (去除空白字元)... ---")
try:
    # 定義所有需要檢查 '是'/'否' 的欄位
    columns_to_clean = [
        '週一', '週二', '週三', '週四', '週五', '週六', '週日',
        '早餐', '午餐', '下午茶', '晚餐'
    ]

    # 確保所有欄位都存在，如果不存在則忽略
    existing_columns_to_clean = [col for col in columns_to_clean if col in df_restaurants.columns]

    # 對所有 '是'/'否' 欄位進行 .str.strip()，去除前後空白
    for col in existing_columns_to_clean:
        # 轉換為字串並去除空白
        df_restaurants[col] = df_restaurants[col].astype(str).str.strip()

        # (可選) 您也可以在這裡統一 '是'/'否' 的大小寫或同義字
        # 例如: .replace(['Y', 'yes'], '是')

    # 清理 '價位' 欄位 (如果也可能含有空白)
    if '價位' in df_restaurants.columns:
        df_restaurants['價位'] = df_restaurants['價位'].astype(str).str.strip()

    print(f"--- 資料清理完成 (處理了 {len(existing_columns_to_clean)} 個欄位) ---")

except Exception as e:
    print(f"!!! 資料清理時發生錯誤: {e}")

--- 成功從 '餐廳列表' 載入 20 筆資料 ---
             店家名稱    招牌菜    價位  \
0      223關東煮A11店    關東煮  200↓   
1        迪化街純正麥芽餅    麥芽餅  100↓   
2       小旺號 - 公館店    炒泡麵  200↓   
3    30 ice cream    冰淇淋  100↓   
4  好正商行 HAO ZHENG  台啤冰淇淋  200↓   

                                          Google map         lat          lng  \
0  https://maps.app.goo.gl/7giRMaENghKyKNmp6?g_st=ac  25.0366358  121.5629753   
1  https://maps.app.goo.gl/3c8EAhDGAqj23Htr9?g_st=ac   25.054271   121.510538   
2  https://maps.app.goo.gl/XdQs4cy67KXsV6v86?g_st=ac  25.0141655  121.5341438   
3  https://maps.app.goo.gl/sHJLsHR9GaxHpPGu8?g_st=ac   25.049268  121.5130451   
4  https://maps.app.goo.gl/qq3E1D9jsAasfdPD6?g_st=ac  25.0446061  121.5457779   

  週一 週二 週三 週四 週五 週六 週日 早餐 午餐 下午茶 晚餐  
0  否  否  否  否  否  否  否  否  否   否  否  
1  是  是  是  是  是  否  是  否  是   是  是  
2  是  是  是  是  是  是  是  是  是   是  是  
3  否  是  是  否  是  是  是  否  否   是  是  
4  是  是  是  是  是  是  是  否  否   是  是  

--- 正在清理資料 (去除空白字元)... ---
--- 資料清理完成 (處理了 11 個欄位)

In [42]:
# --- 5. 產生並顯示 Folium 地圖 (在 Colab 輸出) ---

try:
    tz = gettz(TIMEZONE) # 使用您在 4. 區塊定義的 TIMEZONE
    today_weekday_int = dt.now(tz=tz).weekday() # 0=週一, ..., 6=週日
    weekday_map = { 0: '週一', 1: '週二', 2: '週三', 3: '週四', 4: '週五', 5: '週六', 6: '週日' }
    today_weekday_column = weekday_map[today_weekday_int]

    # 檢查這個欄位是否真的存在於 B_df
    if today_weekday_column not in df_restaurants.columns:
        print(f"!!! 地圖錯誤: 您的 Google Sheet 缺少 '{today_weekday_column}' 欄位。地圖顏色將不準確。")
        today_weekday_column = None # 設為 None，讓後續判斷都走 'gray'
    else:
        print(f"--- 地圖顏色依據: {today_weekday_column} (今天) ---")

except Exception as e:
    print(f"!!! 獲取今日星期失敗: {e}")
    today_weekday_column = None # 設為 None 以便安全失敗


# 使用程式碼一中的地圖中心點
lat_center = 25.027472815739536
lng_center = 121.52993850781064
m = folium.Map(location=(lat_center, lng_center), zoom_start=15)

folium.TileLayer(
    'CartoDB dark_matter',
    name='暗色模式'
).add_to(m)

folium.TileLayer(
    'Esri World Imagery',
    name='衛星影像'
).add_to(m)

folium.TileLayer(
    'CartoDB positron',
    name='簡潔模式 (亮色)'
).add_to(m)

# 根據您 Gradio 篩選器中的價位來命名
group_100 = folium.FeatureGroup(name='價位: 100↓', show=True)
group_200 = folium.FeatureGroup(name='價位: 200↓', show=True)
group_300 = folium.FeatureGroup(name='價位: 200↑', show=True)

# 先將圖層組加入地圖
m.add_child(group_100)
m.add_child(group_200)
m.add_child(group_300)

print("\n--- 正在將餐廳位置添加到地圖... ---")
valid_coordinates_list = [] # 用於自動縮放
locations_added = 0
for i in range(0, len(df_restaurants)):
    try:
        # 確保 lat 和 lng 欄位存在且為有效數字
        lat = float(df_restaurants.loc[i, 'lat'])
        lng = float(df_restaurants.loc[i, 'lng'])
        valid_coordinates_list.append([lat, lng]) # 加入列表
    except (KeyError, ValueError, TypeError):
        # 如果 'lat'/'lng' 不存在或格式錯誤，則跳過
        print(f"警告: '{df_restaurants.loc[i, '餐廳名稱']}' 座標無效或缺失，跳過此地圖標記。")
        continue

    row_data = df_restaurants.loc[i]

    # 準備彈窗 (Popup) 所需的資料
    try:
        name = row_data.get('店家名稱', 'N/A')
        special = row_data.get('招牌菜', 'N/A')
        price = row_data.get('價位', 'N/A')
        gmap_url = row_data.get('Google map', '#')

        # 準備 營業星期 和 營業餐別
        days_open = []
        day_cols = ['週一', '週二', '週三', '週四', '週五', '週六', '週日']
        for day in day_cols:
            if row_data.get(day) == '是':
                days_open.append(day)

        meals_served = []
        meal_cols = ['早餐', '午餐', '下午茶', '晚餐']
        for meal in meal_cols:
            if row_data.get(meal) == '是':
                meals_served.append(meal)

        # 轉換為字串，如果列表為空則顯示 '未提供'
        days_str = "、".join(days_open) if days_open else "未提供"
        meals_str = "、".join(meals_served) if meals_served else "未提供"

        # 安全檢查 URL
        if not isinstance(gmap_url, str) or not gmap_url.startswith('http'):
            gmap_link = "<span>無 Google Map 連結</span>"
        else:
            gmap_link = f'<a href="{gmap_url}" target="_blank" rel="noopener noreferrer">在 Google 地圖上開啟</a>'

    except Exception as e:
        print(f"警告: 讀取 {row_data.get('店家名稱', '未知店家')} 資料時出錯: {e}")
        continue

    # 建立「豐富彈出視窗 (Enriched Popup)」
    html = f"""
    <div style="font-family: Arial, sans-serif; width: 220px;">
        <h4 style="margin: 5px 0 10px 0; color: #333;">{name}</h4>
        <p style="margin: 3px 0;">
            <b>招牌菜:</b> {special}
        </p>
        <p style="margin: 3px 0 10px 0;">
            <b>價位:</b> {price}
        </p>
        <p style="margin: 3px 0; font-size: 0.9em;">
          <b>營業星期:</b> {days_str}
      </p>
      <p style="margin: 3px 0 10px 0; font-size: 0.9em;">
          <b>供應餐別:</b> {meals_str}
      </p>
        {gmap_link}
    </div>
    """
    # 將 HTML 包裝成 IFrame 和 Popup
    iframe = folium.IFrame(html, width=250, height=180)
    popup = folium.Popup(iframe, max_width=250)

    icon_color = 'gray' # 預設為灰色 (未營業)

    # 檢查 today_weekday_column 是否有效 (在 5.1 設定)
    if today_weekday_column:
        try:
            # 檢查 "今天" (例如 '週一') 的欄位值是否為 '是'
            if row_data[today_weekday_column] == '是':
                icon_color = 'blue' # 藍色 (有營業)

        except KeyError:
            # 如果 row_data 中缺少 '週一' 或 '無提供星期' 欄位，跳過
            pass

    marker = folium.Marker(
        location=[lat, lng],
        tooltip=name, # Tooltip (懸停) 保持簡潔
        popup=popup,  # <--- 使用我們剛建立的豐富彈窗
        icon=folium.Icon(color=icon_color, icon="cutlery", prefix='fa'),
    )

    # 根據「價位」將 Marker 加入對應的「圖層分組」
    if price == '100↓':
        marker.add_to(group_100)
    elif price == '200↓':
        marker.add_to(group_200)
    elif price == '200↑':
        marker.add_to(group_300)
    else:
        continue
    locations_added += 1

print(f"--- 地圖標記完成 (共 {locations_added} 個) ---")

# 動態縮放地圖以包含所有標記
if valid_coordinates_list:
    try:
        # Folium's fit_bounds 需要 [[min_lat, min_lon], [max_lat, max_lon]]
        lats = [p[0] for p in valid_coordinates_list]
        lons = [p[1] for p in valid_coordinates_list]
        bounds = [[min(lats), min(lons)], [max(lats), max(lons)]]

        print(f"--- 正在動態調整地圖邊界以包含所有 {len(valid_coordinates_list)} 個標記 ---")
        m.fit_bounds(bounds, padding=(30, 30)) # (padding 可選)
    except Exception as e:
        print(f"--- 警告: 動態縮放失敗: {e} ---")
else:
    print("--- 警告: 找不到任何有效座標，地圖將使用預設中心 ---")

MiniMap(
    tile_layer='CartoDB positron', # 小地圖的樣式
    position='bottomleft',
    toggle_display=True
).add_to(m)

folium.LayerControl().add_to(m)

# 將地圖轉換為 HTML 字串，以便嵌入 Gradio
map_html = m._repr_html_()
print("--- 地圖已轉換為 HTML ---")

print("--- 地圖顯示完畢。Gradio 應用即將啟動... ---\n")

--- 地圖顏色依據: 週日 (今天) ---

--- 正在將餐廳位置添加到地圖... ---
--- 地圖標記完成 (共 20 個) ---
--- 正在動態調整地圖邊界以包含所有 20 個標記 ---
--- 地圖已轉換為 HTML ---
--- 地圖顯示完畢。Gradio 應用即將啟動... ---



In [43]:
# --- 6. 核心功能函數 (Gradio App) ---

def filter_and_recommend(df, price_filter, meal_filter_list, day_filter_value):
    # 動態取得 "星期N" 欄位名稱
    if day_filter_value == '只看今日營業':
        tz = gettz("Asia/Taipei")
        today_weekday_int = dt.now(tz=tz).weekday() # 0=週一, ..., 6=週日
        weekday_map = { 0: '週一', 1: '週二', 2: '週三', 3: '週四', 4: '週五', 5: '週六', 6: '週日' }
        day_column_to_check = weekday_map[today_weekday_int]

        # 檢查欄位是否存在
        if day_column_to_check not in df.columns:
            print(f"警告: 找不到 {day_column_to_check} 欄位。營業日篩選已停用。")
            day_column_to_check = None # 停用篩選
    else:
        # 使用者選擇了特定日期, e.g., '週一'
        day_column_to_check = day_filter_value

        if day_column_to_check not in df.columns:
            print(f"警告: 找不到 {day_column_to_check} 欄位。營業日篩選已停用。")
            day_column_to_check = None # 停用篩選

    # 準備餐別篩選
    # meal_filter_list 會是 e.g., ['午餐', '晚餐'] 或 None
    apply_meal_filter = bool(meal_filter_list and len(meal_filter_list) > 0)


    """根據使用者輸入篩選並隨機推薦 3 家餐廳"""

    # 轉換篩選條件為列表
    valid_prices = ['100↓', '200↓', '200↑'] # [新增] 先定義有效的價位

    if price_filter and len(price_filter) > 0:
        # 情況1: 使用者有勾選，使用使用者的選擇
        price_list = price_filter
    else:
        # 情況2: 使用者未勾選 (留空)，預設為選取所有 *有效* 價位
        price_list = valid_prices # [修改]

    # 套用基本篩選 (價位)
    filtered_df = df[
        df['價位'].isin(price_list)
    ].copy()

    # 動態套用 "餐別" 篩選
    if apply_meal_filter and not filtered_df.empty:
        mask = pd.Series(True, index=filtered_df.index)
        for meal_col in meal_filter_list:
            if meal_col in filtered_df.columns:
                mask = mask & (filtered_df[meal_col] == '是')
            else:
                print(f"警告: 找不到餐別欄位 {meal_col}")
                # 如果連一個欄位都找不到，AND 邏輯下應篩選掉所有
                mask = pd.Series(False, index=filtered_df.index)
                break # 提前中斷

        filtered_df = filtered_df[mask]

    # 動態套用 "營業日" 篩選
    # (day_column_to_check 是在步驟 1 取得的)
    if day_column_to_check and not filtered_df.empty:
        # 我們在 4.1 已清理過資料，所以 '是' 是可靠的
        filtered_df = filtered_df[filtered_df[day_column_to_check] == '是']

    if filtered_df.empty:
        return pd.DataFrame(), "沒有找到符合條件的餐廳。", ""

    # 隨機抽取 3 家，不足 3 家則全數取出
    n_recommend = min(3, len(filtered_df))
    recommended_df = filtered_df.sample(n=n_recommend, replace=False)

    # 準備 AI 摘要的輸入字串
    ai_input = "、".join([
        f"{i+1}. {row['店家名稱']} - 招牌菜：{row['招牌菜']}（價位：{row['價位']}）"
        for i, row in recommended_df.iterrows()
    ])


    return recommended_df, ai_input


async def generate_summary(ai_input):
    """呼叫 LLM 產生決策建議摘要 (使用 gemini-2.5-pro)"""
    if not ai_input:
        return "請先點擊「開始推薦」以產生餐廳清單。"


    system_prompt = (
        "您是一位幽默且極具效率的用餐決策顧問。您的任務是根據提供的餐廳清單，"
        "分析其優缺點（如價格、招牌菜、特色）並總結，幫助使用者在 3 家中快速做出決定。"
        "請以輕鬆口吻，用 **繁體中文** 撰寫一個約 **200 字** 的決策建議摘要，不超過 500 字。"
        "請避免使用 Markdown 標題和編號清單。"
    )

    user_query = f"這是三家推薦的餐廳清單：{ai_input}。請綜合考量其價格、招牌菜和特色，為我提供一份快速決策建議。"
    combined_prompt = f"{system_prompt}\n\n{user_query}"

    try:
        # 1. 呼叫 API，傳入合併後的 prompt 和 timeout 參數
        response = await asyncio.to_thread(
            model.generate_content,
            combined_prompt,
            request_options={"timeout": 120}
        )

        # 2. 仿照您成功的範例，檢查 response.text
        if hasattr(response, 'text') and response.text:
            advice = response.text.strip()
            # (可選) 移除 Markdown 符號
            advice = advice.replace("#", "").replace("*", "")
            return advice
        else:
            return "⚠️ Gemini 分析完成，但沒有回傳文字內容。"

    except Exception as e:
        # 3. 回傳確切的 API 錯誤
        error_msg = f"呼叫 Gemini API 失敗，請檢查 API 金鑰或配額：{e}"
        return error_msg

# 介面更新函數
async def wrapper_recommend(price_filter, meal_filter_list, day_filter_value):
    """Gradio 主要流程：篩選 -> 推薦 -> 呼叫 AI 摘要"""

    # 1. 篩選與推薦
    recommended_df, ai_input = filter_and_recommend(
        df_restaurants, price_filter, meal_filter_list, day_filter_value
    )

    # 2. 呼叫 AI 摘要
    ai_summary = await generate_summary(ai_input)

    # 3. 準備給 Gradio 輸出
    # 將推薦結果轉換為 Dataframe
    display_df = recommended_df[["店家名稱", "招牌菜", "價位", "Google map"]].reset_index(drop=True)

    # Gradio 的 DataFrame 支援 Markdown 連結 [Link Text](URL)
    if 'Google map' in display_df.columns:
        display_df['Google map'] = display_df.apply(
            lambda row: f"[{row['店家名稱']} 等著您！]({row['Google map']})"
                        if pd.notna(row['Google map']) and str(row['Google map']).startswith('http')
                        else '無連結',
            axis=1
        )

    # 處理空結果
    if display_df.empty:
        return pd.DataFrame(), "沒有找到符合條件的餐廳。", "（無建議）"

    return display_df, ai_summary

In [44]:
# --- 7. Gradio 介面配置 ---

# 設定 Gradio 介面元件
with gr.Blocks(title="用餐決策器") as demo:
    gr.Markdown(
        """
        # 🍽️ 用餐決策器
        ### 目標：從餐廳清單中篩選，並隨機推薦 3 家，最後由 AI 助您快速決策！
        """
    )

    with gr.Tabs():
      with gr.TabItem("🍽️ 餐廳推薦器"):

        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("## 條件篩選")
                price_select = gr.CheckboxGroup(
                    ['100↓', '200↓', '200↑'],
                    label="價位過濾 (可複選)",
                    value=None, # **修改點：預設為空 (None 或 [])**
                    info="(以最高價格為分類依據)"
                )
                meal_select = gr.CheckboxGroup(
                    label="餐別過濾 (可複選)",
                    choices=['早餐', '午餐', '下午茶', '晚餐'],
                    value=None, # 預設不過濾
                    info="留空表示不過濾"
                )
                day_select = gr.Dropdown(
                    label="營業日篩選",
                    choices=['只看今日營業', '週一', '週二', '週三', '週四', '週五', '週六', '週日'],
                    value='只看今日營業',
                    info="選擇特定日期，或自動判斷今天"
                )
                recommend_btn = gr.Button("🚀 開始推薦")

            with gr.Column(scale=2):
                gr.Markdown("## 推薦清單 (隨機 3 家)")
                output_dataframe = gr.DataFrame(
                    headers=["店家名稱", "招牌菜", "價位", "Google map"],
                    datatype=["str", "str", "str", "markdown"],
                    label="隨機推薦結果"
                )

                gr.Markdown("## 🤖 AI 決策建議")
                output_summary = gr.Markdown(
                    label="AI 決策顧問報告",
                    value="AI 將根據推薦結果，總結優缺點，幫助您快速做出選擇。"
                )
      with gr.TabItem("🗺️ 地圖總覽"):
        gr.Markdown("### 餐廳地理位置總覽\n在地圖上縮放並點擊圖示查看詳情。")
        gr.HTML(value=map_html, label="餐廳地圖")

    # 定義點擊按鈕後的行為 (使用異步包裝函數)
    recommend_btn.click(
        fn=wrapper_recommend,
        inputs=[
            price_select,
            meal_select,
            day_select
        ],
        outputs=[output_dataframe, output_summary]
    )

    gr.Markdown(
        """
        ---
        **[開發者備註]** AI 摘要已設定為使用 `gemini-2.5-pro` 呼叫。請確保您已執行以下步驟：
        1.  在 Colab 中安裝 `google-genai` 等必要套件。
        2.  在 Colab Secrets 中設定名為 `gemini` 的 API 金鑰。
        """
    )

In [45]:
# --- 8. 運行 Gradio 應用 ---
if __name__ == "__main__":
    # 在 Colab 或一般環境中運行
    print("Gradio 應用正在啟動...")
    # inbrowser=True 在 Colab 中有時效果不好，
    # share=True 會產生一個公開連結，在 Colab 中更穩定
    # debug=True 可以在錯誤時提供更詳細的輸出
    demo.launch(debug=True, share=True)

Gradio 應用正在啟動...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://70a81343e402701c36.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 2027.64ms


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://70a81343e402701c36.gradio.live
